In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import BaggingClassifier

import os

from algorithm_wrapper import ModelWrapper

In [ ]:
# Defina os diretórios
source_dir = 'selected_images_dog_cat'

uploaded_images = []
# Percorra os arquivos no diretório de origem
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    source_file = source_file.replace('\\', '/')
    uploaded_images.append(source_file)

print(uploaded_images)

In [ ]:
# Define a função para determinar a classe com base no nome do arquivo
def determine_class(filename):
    if 'boxer' in filename or 'wheaten_terrier' in filename:
        return 'dog'
    elif 'Siamese' in filename or 'Abyssinian' in filename:
        return 'cat'
    else:
        return 'unknown'

# Cria uma nova coluna 'Class' no dataset
classes = [determine_class(filename) for filename in uploaded_images]

# Exibe as classes
print(classes.count('dog'))
print(classes.count('cat'))
print(len(classes))
print(classes)

In [ ]:
import os
import json

# Define the filename
filename = 'metrics.json'

# Check if the file exists
if not os.path.exists(filename):

    # Create and write the initial content to the file
    open(filename, 'w').close()

    print(f"File '{filename}' created with initial content.")
else:
    print(f"File '{filename}' already exists.")


In [ ]:
filename = 'features.json'

features_dict = {}

try:
    with open('features.json', 'r') as f:
        features_dict = json.load(f)
except Exception:
    print("Maybe you didn't run 'extrair_features.ipynb' first, it should be run before this one.")

In [ ]:
metrics_dict = {}

bagging_models = [
    *[BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=3), n_estimators=i) for i in [10,20,30]],
    *[BaggingClassifier(estimator=GaussianNB(), n_estimators=i) for i in [10,20,30]],
    *[BaggingClassifier(estimator=MLPClassifier(activation="relu",
                                                hidden_layer_size=(100,50),
                                                learning_rate_init=0.0001,
                                                max_iter=500,
                                                solver="sgd"), n_estimators=i) for i in [10,20,30]],
    *[BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=3), n_estimators=i, max_features=0.5) for i in [10,20,30]],
    *[BaggingClassifier(estimator=GaussianNB(), n_estimators=i, max_features=0.5) for i in [10,20,30]],
    *[BaggingClassifier(estimator=MLPClassifier(activation="relu",
                                                hidden_layer_size=(100,50),
                                                learning_rate_init=0.0001,
                                                max_iter=500,
                                                solver="sgd"), n_estimators=i, max_features=0.5) for i in [10,20,30]],
]

# Iterate through the dictionary to get both the name and the variable
for feature_name, feature_data in features_dict.items():
    for bagging_model in bagging_models:
        model_wrapper = ModelWrapper(bagging_model, feature_data, classes)
        print(f"Training model with feature set: {feature_name}")
        model_wrapper.model_train('kfold')
        metrics_dict.update({f"{feature_name}_kfold_k={i}": model_wrapper.metrics})
        model_wrapper.model_train('holdout')
        metrics_dict.update({f"{feature_name}_holdout_k={i}": model_wrapper.metrics})

with open('./metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=4)

In [ ]:
metrics_dict